In [9]:
import pandas as pd
import glob
import numpy as np
import datetime
import logging

In [11]:
start_running=str(datetime.datetime.now())
start_running

'2018-05-02 11:11:45.963745'

In [12]:
x=pd.read_csv("/home/jian/Projects/Big_Lots/Newspaper/BL_combined newspaper final_JL_20180501.csv")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
x.shape

(1244533, 15)

In [18]:
neg=x[x['adjusted_circ_with_list']<0]
neg.to_excel("/home/jian/Projects/Big_Lots/Newspaper/Negative Value_JL_20180501.xlsx",index=False)

In [2]:
folder="/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/"

In [3]:
file_list=glob.glob(folder+"*")
len(file_list)

58

In [4]:
# Check unique extension
set([x[len(x)-5:len(x)] for x in file_list])

{'.xlsx'}

In [5]:
all_columns=[]
for file in file_list:
    df=pd.read_excel(file,sheetname="to_read")
    all_columns=list(set(all_columns+df.columns.tolist()))

In [6]:
all_columns

['productname',
 'dayname',
 'zips',
 'productid',
 'storeid',
 'zonesetname',
 'totalcirc',
 'zoneid',
 'projectname']

In [7]:
Selected_Col=['projectname','storeid','productname','productid','dayname','zips','totalcirc']
output_combined_all_date=pd.DataFrame()
check_df=pd.DataFrame()
finished=[]
total_circ=0
for file in file_list:

    date=file.split("/")[len(file.split("/"))-1].split(" ")[0]+" "+file.split("/")[len(file.split("/"))-1].split(" ")[1]+" "+\
    file.split("/")[len(file.split("/"))-1].split(" ")[2]
    date=date.replace("Sept","Sep")

    if len(date.split(" ")[0])==3:
        date=datetime.datetime.strptime(date,"%b %d %Y").date()
    else:
        date=datetime.datetime.strptime(date,"%B %d %Y").date()
    df=pd.read_excel(file,sheetname="to_read",dtype=str,na_values=['NULL',""," "])
    df=df[Selected_Col]
    df['totalcirc']=df['totalcirc'].astype(float)
    df['keys']=df['storeid']+"_"+df['productid']
    df['Date']=date
    check_df=check_df.append(df)
    total_circ=total_circ+df['totalcirc'].sum()

    df_combine_date=pd.DataFrame()
    for key in df['keys'].unique():
        df_product_store=df[df['keys']==key]
        df_product_store_NA=df_product_store[df_product_store['zips']=='nan']
        df_product_store_NA.reset_index(inplace=True)
        del df_product_store_NA['index']
        df_product_store_zip=df_product_store[df_product_store['zips']!='nan']
        df_product_store_zip.reset_index(inplace=True)
        del df_product_store_zip['index']        

        if len(df_product_store_NA)==0:
            df_product_store_zip=df_product_store.groupby(['storeid','productname','productid','dayname','zips'])['totalcirc'].sum().to_frame()
            df_product_store_zip.reset_index(inplace=True)
            df_product_store_zip['circ_proportion_of_Null']=0
            df_product_store_zip['adjusted_circ_about_Null']=df_product_store_zip['totalcirc']
            
        elif len(df_product_store_zip)==0:
            # print(str(date),key,"All Null Zips")
            df_product_store_zip=df_product_store.groupby(['storeid','productname','productid','dayname','zips'])['totalcirc'].sum().to_frame()
            df_product_store_zip.reset_index(inplace=True)
            df_product_store_zip['circ_proportion_of_Null']=1
            df_product_store_zip['adjusted_circ_about_Null']=df_product_store_zip['totalcirc']
            
        elif len(df_product_store_NA)>=1:
            na_zip_circ=df_product_store_NA['totalcirc'].sum()
            df_product_store_zip=df_product_store_zip.groupby(['storeid','productname','productid','dayname','zips'])['totalcirc'].sum().to_frame()
            df_product_store_zip.reset_index(inplace=True)
            df_product_store_zip['circ_proportion_of_Null']=df_product_store_zip['totalcirc'].apply(lambda x:x/sum(df_product_store_zip['totalcirc']))
            df_product_store_zip['adjusted_circ_about_Null']=df_product_store_zip['totalcirc']+df_product_store_zip['circ_proportion_of_Null']*na_zip_circ
        df_combine_date=df_combine_date.append(df_product_store_zip)

    df_combine_date['Date']=date
    finished=finished+[str(date)]
    print(str(date),"Done",len(finished))

    output_combined_all_date=output_combined_all_date.append(df_combine_date) 
# output_combined_all_date.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/Ongoing/Newspaper/output/combined all newspaper.csv",index=False)


'''
# Check Dates code

    date_in_projectname=df['projectname'].apply(lambda x: x.replace("Sept","Sep")).apply(lambda x: x.split(" ")[0]+" "+x.split(" ")[1]+" "+x.split(" ")[2]).unique().tolist()
    date_in_file_list=[]
    for date_A in date_in_projectname:

        if len(date_A.split(" ")[0])==3:
            date_A=datetime.datetime.strptime(date_A,"%b %d %Y").date()
            date_in_file_list=list(set(date_in_file_list+[date_A]))
        else:
            date_A=datetime.datetime.strptime(date_A,"%B %d %Y").date()
            date_in_file_list=list(set(date_in_file_list+[date_A]))
            
    if len(date_in_file_list)==1:
        if date!=date_in_file_list[0]:
            print(str(date),": ","Date different from the file name")
    else:
        print(str(date),": ","More than 1 date in the file")
    del df['projectname']    
'''


'\n# Check Dates code\n\n    date_in_projectname=df[\'projectname\'].apply(lambda x: x.replace("Sept","Sep")).apply(lambda x: x.split(" ")[0]+" "+x.split(" ")[1]+" "+x.split(" ")[2]).unique().tolist()\n    date_in_file_list=[]\n    for date_A in date_in_projectname:\n\n        if len(date_A.split(" ")[0])==3:\n            date_A=datetime.datetime.strptime(date_A,"%b %d %Y").date()\n            date_in_file_list=list(set(date_in_file_list+[date_A]))\n        else:\n            date_A=datetime.datetime.strptime(date_A,"%B %d %Y").date()\n            date_in_file_list=list(set(date_in_file_list+[date_A]))\n            \n    if len(date_in_file_list)==1:\n        if date!=date_in_file_list[0]:\n            print(str(date),": ","Date different from the file name")\n    else:\n        print(str(date),": ","More than 1 date in the file")\n    del df[\'projectname\']    \n'

In [8]:
total_circ

950699801.49969983

In [8]:
check_df['totalcirc'].sum()

950699801.49970007

In [19]:
output_combined_all_date['adjusted_circ_about_Null'].sum()

950699801.49970019

In [20]:
output_combined_all_date.reset_index(inplace=True)
del output_combined_all_date['index']
output_combined_all_date.to_excel("/home/jian/Projects/Big_Lots/Newspaper/BL_combined newspaper data_JL_20180501.xlsx",index=False)

In [34]:
household=pd.read_csv("/home/jian/Docs/Household and Population/2016/ACS_16_5YR_S1101_with_ann (Household Composition).csv",dtype=str,skiprows=1)
household=household.iloc[:,[0,4]]
household['zip_cd']=household['Id'].apply(lambda x: x[len(x)-5:len(x)])
household=household[['zip_cd','Total; Margin of Error; Total households']]
household.columns=['zip_cd','Households']
household['Households']=household['Households'].fillna(0)
household['Households']=household['Households'].astype(int)

In [24]:
output_combined_all_date['zip_list']=output_combined_all_date['zips'].apply(lambda x: [ zip.strip().zfill(5) for zip in x.split(",")])

In [25]:
output_combined_all_date['zip_count']=output_combined_all_date['zip_list'].apply(lambda x: len(x))

In [26]:
output_combined_all_date_1=output_combined_all_date[output_combined_all_date['zip_count']==1]
output_combined_all_date_M=output_combined_all_date[output_combined_all_date['zip_count']!=1]

In [27]:
output_combined_all_date_1['zip_cd']=output_combined_all_date_1['zips'].str.zfill(5)
output_combined_all_date_1['zip_list_proportion']=1
output_combined_all_date_1['adjusted_circ_with_list']=output_combined_all_date_1['adjusted_circ_about_Null']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
output_combined_all_date_M.reset_index(inplace= True)
del output_combined_all_date_M['index']
output_combined_all_date_M_agg=pd.DataFrame()

for i in range(len(output_combined_all_date_M)):
    df=output_combined_all_date_M.iloc[i:(i+1),:]
    df.reset_index(inplace=True)
    del df['index']
    df_to_app=pd.DataFrame()
    for zip_i in list(set(df['zip_list'][0])):
        df_1_zip=df.copy()
        df_1_zip['zip_cd']=zip_i
        df_to_app=df_to_app.append(df_1_zip)
    df_to_app=pd.merge(df_to_app,household,on="zip_cd",how="left")
    df_to_app['zip_list_proportion']=df_to_app['Households']/df_to_app['Households'].sum()
    df_to_app['adjusted_circ_with_list']=df_to_app['adjusted_circ_about_Null']*df_to_app['zip_list_proportion']
    output_combined_all_date_M_agg=output_combined_all_date_M_agg.append(df_to_app)
    if i % 1000 == 1:
        print(i)

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901


KeyboardInterrupt: 

In [ ]:
output_combined_all_date_M_agg.head(2)

In [ ]:
output_adjusted_final=output_combined_all_date_M_agg.append(output_combined_all_date_1)

In [ ]:
output_adjusted_final.to_csv("/home/jian/Projects/Big_Lots/Newspaper/BL_combined newspaper final_JL_20180424.csv",index=False)

In [2]:
x=pd.read_csv("/home/jian/Projects/Big_Lots/Newspaper/BL_combined newspaper final_JL_20180424.csv")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
x.shape

(1244533, 15)

In [11]:
sample_data_list=[
 '2017-11-12',
 '2017-11-19',
 '2017-11-22',
 '2017-11-23',
 '2017-11-26',
 '2017-12-03']

In [12]:
sample=x[x['Date'].isin(sample_data_list)]

In [13]:
sample.to_csv("/home/jian/Projects/Big_Lots/Newspaper/BL_combined newspaper sample.csv",index=False)

In [10]:
sorted(x['Date'].unique())

['2016-08-28',
 '2016-09-04',
 '2016-09-25',
 '2016-10-09',
 '2016-10-23',
 '2016-10-30',
 '2016-11-06',
 '2016-11-13',
 '2016-11-20',
 '2016-11-23',
 '2016-11-24',
 '2016-11-27',
 '2016-12-04',
 '2016-12-11',
 '2016-12-18',
 '2017-01-08',
 '2017-02-05',
 '2017-02-12',
 '2017-02-19',
 '2017-03-05',
 '2017-03-12',
 '2017-03-19',
 '2017-04-02',
 '2017-04-09',
 '2017-04-23',
 '2017-04-30',
 '2017-05-21',
 '2017-05-28',
 '2017-06-11',
 '2017-06-25',
 '2017-07-16',
 '2017-07-30',
 '2017-08-27',
 '2017-09-03',
 '2017-09-24',
 '2017-10-08',
 '2017-10-22',
 '2017-10-29',
 '2017-11-05',
 '2017-11-12',
 '2017-11-19',
 '2017-11-22',
 '2017-11-23',
 '2017-11-26',
 '2017-12-03',
 '2017-12-10',
 '2017-12-17',
 '2018-01-07',
 '2018-02-11',
 '2018-02-18',
 '2018-02-25',
 '2018-03-04',
 '2018-03-18',
 '2018-03-25',
 '2018-04-08',
 '2018-04-15',
 '2018-04-22',
 '2018-04-29']